##### Description des fonctions et des paramètres :

1. **Imports et Configuration de base**
   - Importation des bibliothèques nécessaires : `sys`, `os`, `subprocess`, et des modules spécifiques du projet.
   - Ajout du chemin vers le dossier 'scripts/outils' au `sys.path`.

2. **Paramètres**
   - **Répertoires d'entrée :**
     - `input_tg_dir`: Répertoire contenant les fichiers TextGrid.
     - `input_pitchtier_dir`: Répertoire contenant les fichiers PitchTier.
     - `input_conllu_dir`: Répertoire contenant les fichiers CoNLL-U.
   - **Choix du script IPU à exécuter :**
     - `ipu_script_sppas`: Utiliser SPPAS pour l'extraction des IPU (True ou False).
     - `ipu_script_webrtc`: Utiliser WebRTC VAD pour l'extraction des IPU (True ou False).

3. **Boucle principale**
   - Parcourt chaque sous-dossier dans `input_tg_dir`.
   - Pour chaque sous-dossier, collecte les fichiers .wav et .TextGrid correspondants.
   - Pour chaque paire de fichiers .wav et .TextGrid correspondants :
     - **Extraction des IPU :**
       - Si `ipu_script_sppas` est True, exécute la commande SPPAS pour extraire les IPU.
       - Si `ipu_script_webrtc` est True, utilise la fonction `process_file` du script `webrtcvad_ipu` pour extraire les IPU.
     - **Correction de la durée des silences dans le tier IPU :**
       - Utilise la fonction `correct_silence_duration` pour corriger la durée des silences et supprimer ceux de moins de 0.1s.
     - **Extraction des tokens, syllabes, transcriptions phonétiques et alignements :**
       - Exécute la commande SPPAS pour obtenir les tokens, syllabes, transcriptions phonétiques et alignements.
     - **Obtention des indices des tokens :**
       - Utilise la fonction `get_index_function` pour obtenir les indices des tokens à partir des fichiers CoNLL-U et des fichiers alignés phonétiquement.

##### Requis:
- SPPAS
- Praat
- TextGrid ayant la transcription de l'audio dans un intervalle nommé "trans" (`1-create_trans_tier.ipynb`)
- Fichier .wav de l'audio

##### Résumé :
Le script traite les fichiers .wav et .TextGrid pour extraire les IPU, tokens, syllabes, transcriptions phonétiques et alignements en utilisant SPPAS et WebRTC VAD. Il effectue également des corrections sur les durées des silences et génère les indices des tokens nécessaires pour l'analyse linguistique.

In [1]:
import sys
import os
import subprocess

base_path = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(base_path, 'scripts/outils')))

from modif_ipus_tier import *
from webrtcvad_ipu import process_file
from get_index import *
from tqdm import tqdm

## Paramètres

In [2]:
input_tg_dir = "../data/TG_WAV/" # Directory containing the TextGrid files
input_pitchtier_dir = "../data/PITCHTIER/" # Directory containing the PitchTier files
input_conllu_dir = "../data/SUD_Naija-NSC-master/gold_nongold" # Directory containing the CoNLL-U files

# Chooses the script to execute for the IPUs
ipu_script_sppas = False
ipu_script_webrtc = True

In [9]:
for subdir in tqdm(os.listdir(input_tg_dir)):
    subdir_path = os.path.join(input_tg_dir, subdir)

    # If the subdirectory is a directory
    if os.path.isdir(subdir_path):
        # Lists to store the names of .wav and .TextGrid files
        wav_files = []
        textgrid_files = []

        # Iterate through all the files in the subdirectory
        for file in os.listdir(subdir_path):
            if file.endswith(".wav"):
                wav_files.append(file)
            elif file.endswith("_M.TextGrid") or file.endswith("_MG.TextGrid"):
                textgrid_files.append(file)
        

        # Execute the command for each corresponding file pair
        for wav_file in wav_files:
            # Build the corresponding .TextGrid file name
            textgrid_file = wav_file.replace(".wav", ".TextGrid")

            if textgrid_file in textgrid_files:
                # Construct the paths of the .wav and .TextGrid files
                wav_file_path = os.path.join(subdir_path, wav_file)
                textgrid_file_path = os.path.join(subdir_path, textgrid_file)

                # Command to obtain the IPU
                if ipu_script_sppas:
                    command1 = f"python3 /Users/perrine/Desktop/Stage_2023-2024/SPPAS-4_1/sppas/bin/searchipus.py -I {wav_file_path} -e .TextGrid --min_ipu 0.02 --min_sil 0.1"
                    subprocess.run(command1, shell=True)
                
                elif ipu_script_webrtc:
                    process_file(textgrid_file_path, wav_file_path, limite=0.1)

                ipu_filepath = os.path.join(
                    subdir_path, wav_file.replace(".wav", "-ipus.TextGrid")
                )
                textgrid_filepath = os.path.join(
                    subdir_path, wav_file.replace(".wav", ".TextGrid")
                )
                pitch_path = os.path.join(
                    input_pitchtier_dir, wav_file.replace(".wav", ".PitchTier")
                )

                # Script to modify the IPU tier and correct silence duration, remove silences less than 0.1s
                correct_silence_duration(
                    textgrid_filepath,
                    ipu_filepath,
                    pitch_path,
                    os.path.join(
                        subdir_path, wav_file.replace(".wav", "-ipus.TextGrid")
                    ),
                    limite=0.10,
                )

                # Command to obtain tokens, syllables, phonetic transcriptions, and alignments
                command2 = f"python3 /Users/perrine/Desktop/Stage_2023-2024/SPPAS-4_1/sppas/bin/annotation.py -I {wav_file_path} -I {textgrid_file_path} -l pcm -e .TextGrid --textnorm --phonetize --alignment --syllabify"
                subprocess.run(command2, shell=True)

                # Command to obtain token indices
                conllu_file = os.path.join(
                    input_conllu_dir, wav_file.replace(".wav", ".conllu")
                )
                palign_file = os.path.join(
                    subdir_path, wav_file.replace(".wav", "-palign.TextGrid")
                )

                get_index_function(conllu_file, palign_file)

print("Fin de l'exécution")

  0%|          | 0/82 [00:00<?, ?it/s]/Users/perrine/Desktop/Stage_2023-2024/SPPAS-4_1/sppas/bin/annotation.py:3: SyntaxWarning: invalid escape sequence '\ '
  """
2024-07-22 20:20:01,028 [INFO] Logging redirected to StreamHandler (level=15).
2024-07-22 20:20:01,048 [INFO] Check directories and create if not existing:
2024-07-22 20:20:01,048 [INFO]  - Write access to SPPAS package directory is granted.
2024-07-22 20:20:01,048 [INFO]  - The folder for logs is OK.
2024-07-22 20:20:01,048 [INFO]  - The folder for the workspaces is OK.
2024-07-22 20:20:01,048 [INFO]  - The trash is OK.
2024-07-22 20:20:01,123 [WARNING] Support of images is disabled because it requires image feature.
2024-07-22 20:20:01,128 [WARNING] Support of videos is disabled because it requires video feature.
2024-07-22 20:20:01,173 [WARNING] Face detection is disabled because it requires video feature.
2024-07-22 20:20:01,175 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-07-22 20:20:01,176 [W

------------------------------------------------------------------------

SPPAS   -  Version 4.14
Copyright (C) 2011-2023 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - ../data/TG_WAV/IBA_32/IBA_32_Tori-By-Samuel_MG.wav
 - ../data/TG_WAV/IBA_32/IBA_32_Tori-By-Samuel_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/IBA_32/IBA_32_Tori-By-Samuel_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * * *  Phonetization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/IBA_32/IBA_32_Tori-By-Samuel_MG-token.TextGrid (0 percents)
  => 

  1%|          | 1/82 [00:06<08:13,  6.09s/it]

  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

------------------------------------------------------------------------

See None for details.
Thank you for using SPPAS.
------------------------------------------------------------------------

../data/SUD_Naija-NSC-master/gold_nongold/IBA_32_Tori-By-Samuel_MG.conllu


/Users/perrine/Desktop/Stage_2023-2024/SPPAS-4_1/sppas/bin/annotation.py:3: SyntaxWarning: invalid escape sequence '\ '
  """
2024-07-22 20:20:07,111 [INFO] Logging redirected to StreamHandler (level=15).
2024-07-22 20:20:07,128 [INFO] Check directories and create if not existing:
2024-07-22 20:20:07,129 [INFO]  - Write access to SPPAS package directory is granted.
2024-07-22 20:20:07,129 [INFO]  - The folder for logs is OK.
2024-07-22 20:20:07,129 [INFO]  - The folder for the workspaces is OK.
2024-07-22 20:20:07,129 [INFO]  - The trash is OK.
2024-07-22 20:20:07,208 [WARNING] Support of images is disabled because it requires image feature.
2024-07-22 20:20:07,212 [WARNING] Support of videos is disabled because it requires video feature.
2024-07-22 20:20:07,240 [WARNING] Face detection is disabled because it requires video feature.
2024-07-22 20:20:07,243 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-07-22 20:20:07,244 [WARNING] Person face identification is 

...

------------------------------------------------------------------------

SPPAS   -  Version 4.14
Copyright (C) 2011-2023 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - ../data/TG_WAV/IBA_40/IBA_40_Christ-Passion-Prologue_MG.wav
 - ../data/TG_WAV/IBA_40/IBA_40_Christ-Passion-Prologue_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/IBA_40/IBA_40_Christ-Passion-Prologue_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * * *  Phonetization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/IBA_40/IBA_40_Christ-Passion-Prologue_

 99%|█████████▉| 81/82 [07:17<00:03,  3.74s/it]/Users/perrine/Desktop/Stage_2023-2024/SPPAS-4_1/sppas/bin/annotation.py:3: SyntaxWarning: invalid escape sequence '\ '
  """
2024-07-22 20:27:18,726 [INFO] Logging redirected to StreamHandler (level=15).
2024-07-22 20:27:18,743 [INFO] Check directories and create if not existing:
2024-07-22 20:27:18,743 [INFO]  - Write access to SPPAS package directory is granted.
2024-07-22 20:27:18,743 [INFO]  - The folder for logs is OK.
2024-07-22 20:27:18,743 [INFO]  - The folder for the workspaces is OK.
2024-07-22 20:27:18,743 [INFO]  - The trash is OK.
2024-07-22 20:27:18,809 [WARNING] Support of images is disabled because it requires image feature.
2024-07-22 20:27:18,817 [WARNING] Support of videos is disabled because it requires video feature.
2024-07-22 20:27:18,848 [WARNING] Face detection is disabled because it requires video feature.
2024-07-22 20:27:18,850 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-07-22 20:27:

------------------------------------------------------------------------

SPPAS   -  Version 4.14
Copyright (C) 2011-2023 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - ../data/TG_WAV/LAG_07/LAG_07_Johns-Biography_MG.wav
 - ../data/TG_WAV/LAG_07/LAG_07_Johns-Biography_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/LAG_07/LAG_07_Johns-Biography_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * * *  Phonetization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Gold_Auto/data/TG_WAV/LAG_07/LAG_07_Johns-Biography_MG-token.TextGrid (0 percents)
 

100%|██████████| 82/82 [07:22<00:00,  5.39s/it]

  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

------------------------------------------------------------------------

See None for details.
Thank you for using SPPAS.
------------------------------------------------------------------------

../data/SUD_Naija-NSC-master/gold_nongold/LAG_07_Johns-Biography_MG.conllu
Fin de l'exécution
